# Hilbert-Huang Transform (HHT) Analysis Tutorial

This tutorial explains the procedure for **Hilbert-Huang Transform (HHT)** analysis using **gwexpy**.
HHT is well-suited for analyzing nonlinear and non-stationary signals, and is a powerful method for extracting local frequency variations (instantaneous frequency) that cannot be captured by conventional Fourier transform or wavelet transform.

HHT consists of two main steps:
1.  **Empirical Mode Decomposition (EMD)**: Decomposes the signal into Intrinsic Mode Functions (IMFs).
2.  **Hilbert Spectral Analysis**: Performs Hilbert transform on each IMF to calculate instantaneous amplitude and instantaneous frequency.

**Note**: To use this feature, the `PyEMD` (EMD-signal) package must be installed.
```bash
pip install EMD-signal
```

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from gwexpy.timeseries import TimeSeries

# Check PyEMD availability
try:
    import PyEMD

    print(f"{PyEMD.__name__} is installed and ready.")
except ImportError:
    raise ImportError(
        "This tutorial requires 'PyEMD' (EMD-signal). Please run: pip install EMD-signal"
    )

## 1. Creating Simulation Data

To verify the effectiveness of EMD, we will create a non-stationary synthetic signal with the following characteristics:
1.  **Low-frequency trend**: 5 Hz sine wave
2.  **Chirp signal**: Component with frequency increasing over time (~100 Hz)
3.  **Linear trend**: Linear drift

In [ ]:
# Time axis (1 second, 1000 samples)
t = np.linspace(0, 1, 1000)
dt = t[1] - t[0]

# Signal components
s1 = 0.5 * np.sin(2 * np.pi * 5 * t)  # Low frequency
s2 = 1.0 * np.sin(2 * np.pi * 80 * t * (1 + 0.5 * t))  # Chirp (80Hz -> 120Hz)
trend = 2.0 * t  # Linear trend

# Create TimeSeries object
data = TimeSeries(s1 + s2 + trend, dt=dt, unit="V", name="Simulation Data")

# Plot
plot = data.plot(title="Original Simulation Signal")
plot.show()

## 2. Empirical Mode Decomposition (EMD)

The `TimeSeries` object in `gwexpy` can perform decomposition directly through the `emd()` method.
The `emd()` method functions as a `PyEMD` wrapper and returns the result as a `TimeSeriesDict`.

Here we use basic **EMD**, but **EEMD (Ensemble EMD)** is also optionally available.

In [ ]:
# Execute EMD
# Setting method='eemd' will run Ensemble EMD (takes longer computation time)
imfs = data.emd(method="emd")

print("Extracted IMFs:", list(imfs.keys()))

# Plot all IMFs
plot = imfs.plot(figsize=(10, 12), sharex=True, title="Intrinsic Mode Functions (IMFs)")
plot.show()

Looking at the plot, you can see that the signal has been decomposed into modes (IMFs) for different frequency bands.
*   **IMF 0**: Highest frequency component (part of chirp signal or noise)
*   **IMF 1, 2...**: Lower frequency components
*   **Residual**: Residue (trend component)

---

## 3. Hilbert Spectral Analysis

Since each IMF has properties close to a "single component," physically meaningful **Instantaneous Frequency** and **Instantaneous Amplitude** can be defined by applying the Hilbert transform.

The `TimeSeries` in `gwexpy` provides the following methods:
*   `.hilbert()`: Computes the analytic signal
*   `.instantaneous_frequency()`: Computes instantaneous frequency (derivative of phase)
*   `.envelope()`: Computes envelope (instantaneous amplitude)

In [ ]:
# Dictionary to store analysis results
hht_results = {}

plt.figure(figsize=(10, 6))

# Analyze the first few IMFs
for key in list(imfs.keys())[:3]:  # Loop over first 3 IMFs
    imf = imfs[key]

    # Calculate instantaneous frequency
    # The smooth parameter can smooth out noise from differentiation
    inst_freq = imf.instantaneous_frequency(smooth=10)

    # Calculate instantaneous amplitude (envelope)
    inst_amp = imf.envelope()

    hht_results[key] = {"freq": inst_freq, "amp": inst_amp}

    # Plot time variation of instantaneous frequency
    plt.plot(inst_freq.times.value, inst_freq.value, label=f"{key} Freq")

plt.title("Instantaneous Frequency of IMFs")
plt.xlabel("Time [s]")
plt.ylabel("Frequency [Hz]")
plt.ylim(0, 150)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Hilbert Spectrum

Finally, we combine these instantaneous frequencies and amplitudes to draw the **Hilbert Spectrum**.
This represents the energy distribution on the time-frequency plane and is equivalent to a Fourier transform-based spectrogram, but has the characteristic of being less subject to time-frequency resolution limitations (uncertainty principle).

Here we use a scatter plot that plots each sample's (time, frequency) as a point, with color representing amplitude.

In [ ]:
plt.figure(figsize=(10, 6))

for key, res in hht_results.items():
    t_vals = res["freq"].times.value
    f_vals = res["freq"].value
    a_vals = res["amp"].value

    # Filter because frequency can be negative or extremely large
    mask = (f_vals > 0) & (f_vals < 200)

    plt.scatter(
        t_vals[mask],
        f_vals[mask],
        c=a_vals[mask],
        cmap="viridis",
        s=5,
        alpha=0.7,
        vmin=0,
        vmax=1.0,
    )

plt.colorbar(label="Amplitude [V]")
plt.xlabel("Time [s]")
plt.ylabel("Frequency [Hz]")
plt.title("Hilbert Spectrum (Scatter Plot)")
plt.ylim(0, 150)
plt.grid(True, alpha=0.3, linestyle="--")
plt.show()

### Interpretation of Results
In the Hilbert spectrum, you can very clearly see the chirp signal's frequency increasing linearly over time (80Hz -> 120Hz).
The low-frequency IMF appears as a line at the bottom.

Thus, HHT is effective for detailed analysis of the non-stationary behavior of signals.